## 주피터 노트북 환경에서 Kubeflow Fairing 사용하기

Kubeflow 주피터 노트북 환경에서 Kubeflow Fairing을 사용해보도록 하겠습니다. mnist 숫자를 분류하는 간단한 모델을 컨테이너 이미지로 빌드하고, Kubeflow 클러스터에 job 형태로 배포해서 학습하는 것을 하도록 하겠습니다.
### 1). “notebook” 전처리기를 사용해서 모델 학습하기
주피터 노트북 사용자 인터페이스의 메뉴에서 File > New > Notebook 을 클릭하여 노트북 환경에서 새 노트북을 시작하십시오.

노트북 컬럼에 모델 코드와 faring 코드를 입력하겠습니다.

다음은 mnist 숫자를 분류하는 모델을 텐서플로우 케라스로 작성한 코드입니다.


In [16]:
import os

import tensorflow as tf
import numpy as np

def train():
    print("TensorFlow version: ", tf.__version__)

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Training...")
    training_history = model.fit(x_train, y_train, epochs=5)

    print("Average test loss: ", np.average(training_history.history['loss']))

다음은 fairing 코드 입니다.

In [2]:
def fairing_run():
    CONTAINER_REGISTRY = 'kangwoo'
    namespace = 'admin'
    job_name = f'mnist-job-{uuid.uuid4().hex[:4]}'
    fairing.config.set_builder('append', registry=CONTAINER_REGISTRY, image_name="mnist-simple",
                               base_image="tensorflow/tensorflow:2.1.0-py3")
    fairing.config.set_deployer('job', namespace=namespace, job_name=job_name, cleanup=False, stream_log=True)
    fairing.config.run()

특이하게도 전처리기를 설정하는 set_preprocessor() 구문이 없습니다. 노트북에서는 별도의 전처리기를 설정하지 않으면 “notebook” 전처리기가 사용됩니다. “notebook” 전처리기는 노트북의 파이썬 코드 셀을 파이썬 파일로 변경해 줍니다.

다음은 환경 변수에 따라 모델 훈련을 실행하거나, fairing을 실행하는 부분입니다.

In [3]:
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        import uuid
        from kubeflow import fairing
        fairing_run()
    else:
        train()

[I 200519 09:46:59 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7fa6982efe80>
[I 200519 09:46:59 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fa6719145c0>
[I 200519 09:46:59 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fa67707a208>
[W 200519 09:46:59 append:50] Building image using Append builder...
[I 200519 09:46:59 base:107] Creating docker context: /tmp/fairing_context_k0n2r7it
[I 200519 09:46:59 converted_notebook:127] Converting Untitled1.ipynb to Untitled1.py
[I 200519 09:46:59 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.1.0-py3'
[W 200519 09:47:02 append:54] Image successfully built in 2.721695505999378s.
[W 200519 09:47:02 append:94] Pushing image kangwoo/mnist-simple:2B6E57FE...
[I 200519 09:47:02 docker_creds_:234] Loading Docker credentials for repository 'kangwoo/mnist-s

V2DiagnosticException: response: {'content-type': 'application/json', 'docker-distribution-api-version': 'registry/2.0', 'www-authenticate': 'Bearer realm="https://auth.docker.io/token",service="registry.docker.io",scope="repository:kangwoo/mnist-simple:pull,push",error="insufficient_scope"', 'date': 'Tue, 19 May 2020 09:47:05 GMT', 'content-length': '242', 'strict-transport-security': 'max-age=31536000', 'status': '401'}
authentication required: [{'Type': 'repository', 'Class': '', 'Name': 'kangwoo/mnist-simple', 'Action': 'pull'}, {'Type': 'repository', 'Class': '', 'Name': 'kangwoo/mnist-simple', 'Action': 'push'}]

### 2). “function” 전처리기를 사용해서 모델 학습하기

주피터 노트북 사용자 인터페이스의 메뉴에서 File > New > Notebook 을 클릭하여 노트북 환경에서 새 노트북을 시작하십시오.

노트북 컬럼에 모델 코드와 faring 코드를 입력하겠습니다.

다음은 mnist 숫자를 분류하는 모델을 텐서플로우 케라스로 작성한 코드입니다.

In [17]:
import os

import tensorflow as tf
import numpy as np

def train():
    print("TensorFlow version: ", tf.__version__)

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Training...")
    training_history = model.fit(x_train, y_train, epochs=5)

    print("Average test loss: ", np.average(training_history.history['loss']))

다음은 fairing 코드 입니다.

In [18]:
def fairing_fn(fn):
    CONTAINER_REGISTRY = 'kangwoo'

    namespace = 'admin'
    job_name = f'mnist-job-{uuid.uuid4().hex[:4]}'


    fairing.config.set_builder('append', registry=CONTAINER_REGISTRY, image_name="mnist-simple",
                               base_image="tensorflow/tensorflow:2.1.0-py3")

    fairing.config.set_deployer('job', namespace=namespace, job_name=job_name, cleanup=False, stream_log=True)

    return fairing.config.fn(fn)

전처리기를 설정하는 set_preprocessor() 구문이 없습니다. 그리고 마지막 줄에 fairing을 실행하는 fairing.config.run() 코드가 사라지고, return fairing.config.fn(fn) 코드가 추가되었습니다. fairing.config.fn(fn) 함수는 넘겨진 fn 함수를 직접 실행해주는 “function” 전처리기가 사용됩니다.

다음은 fairing에서 함수를 넘겨받아 실행하는 부분입니다.

In [19]:
if __name__ == '__main__':
    import uuid
    from kubeflow import fairing
    remote_train = fairing_fn(train)
    remote_train()

[I 200519 09:55:00 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7fa665f7a208>
[I 200519 09:55:00 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fa665f7a320>
[I 200519 09:55:00 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fa665f7a278>
[W 200519 09:55:00 append:50] Building image using Append builder...
[I 200519 09:55:00 base:107] Creating docker context: /tmp/fairing_context_25ter2wf
[W 200519 09:55:00 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200519 09:55:00 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.1.0-py3'
[W 200519 09:55:03 append:54] Image successfully built in 2.780903238999599s.
[W 200519 09:55:03 append:94] Pushing image kangwoo/mnist-simple:CA9FE726...
[I 200519 09:55:03 docker_creds_:234] Loading D

V2DiagnosticException: response: {'content-type': 'application/json', 'docker-distribution-api-version': 'registry/2.0', 'www-authenticate': 'Bearer realm="https://auth.docker.io/token",service="registry.docker.io",scope="repository:kangwoo/mnist-simple:pull,push",error="insufficient_scope"', 'date': 'Tue, 19 May 2020 09:55:05 GMT', 'content-length': '242', 'strict-transport-security': 'max-age=31536000', 'status': '401'}
authentication required: [{'Type': 'repository', 'Class': '', 'Name': 'kangwoo/mnist-simple', 'Action': 'pull'}, {'Type': 'repository', 'Class': '', 'Name': 'kangwoo/mnist-simple', 'Action': 'push'}]